In [108]:
import string
import pandas as pd
from collections import Counter
from pymorphy3 import MorphAnalyzer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
df = pd.read_csv("D:\\ds\\jokes.csv")

Напишите функцию для предобработки текста:
- Удаление пунктуационных знаков (символы возьмите из модуля string)
- Используйте кеш для хранения нормальных форм слов
- Лемматизируйте слова

Запустите функцию на десятом по индексу объекте из датасета и укажите нулевое слово.

In [109]:
# Инициализация лемматизатора
morph = MorphAnalyzer()

# Кэш для хранения нормальных форм слов 
cache = {}

def preprocess_txt(line):
    punctuation = string.punctuation  # пунктуационные знаки из модуля string
    tokens = "".join(i.lower() for i in line.strip() if i not in punctuation).split()
    
    words_lem = []
    for w in tokens:
        if w in cache:
            words_lem.append(cache[w])  # добавление норм. формы из кэша
        else:
            temp_cach = cache[w] = morph.parse(w)[0].normal_form  # получение норм. формы
            words_lem.append(temp_cach)
    
    return ' '.join(words_lem)

# Пример использования (предполагая, что df - ваш DataFrame)
text = df.loc[9, 'text']  # десятый объект (индекс 9)
processed_text = preprocess_txt(text)
first_word = processed_text.split()[0] if processed_text else "Текст пуст"

print(f"Нулевое слово: {first_word}")

Нулевое слово: лететь


1. Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=1.

2. Запустите функцию preprocess_txt на тренировочной и тестовой выборках

3. Соедините слова из тренировочной выборки в единый текст в переменную text

In [27]:
# Разбиение на обучающую и тестовую выборки (80/20)
X_train, X_test = train_test_split(df['text'], test_size=0.2, random_state=1)

# Применяем функцию preprocess_txt к тренировочной выборке
train_processed = X_train.apply(preprocess_txt)

# Применяем функцию preprocess_txt к тестовой выборке
test_processed = X_test.apply(preprocess_txt)

# Функция для фильтрации пустых значений и преобразования в строку
def filter_and_join(text_series):
    # Преобразуем список в строку, игнорируя NaN и пустые значения
    return ' '.join(str(word) for word in text_series.explode() if isinstance(word, str))

# Объединяем все лемматизированные слова из тренировочной выборки в единую строку
text = filter_and_join(train_processed)

# Выводим результат
print(f"Текст из тренировочной выборки (объединенные слова): {text[:500]}...")

Текст из тренировочной выборки (объединенные слова): моргот трындец похоже в жизнь прийтись что то пересматривать gemoroy что случиться моргот нет ты прикинуть я блэкушник с десятилетний стаж моргот обладатель чумовой гроул моргот да когда я говорить собака разбегаться от мой рёв моргот и всё это уйти в никуда сейчас моргот когда по телефон после мой аллла какой то старуха попросить девочка принять факс принимать както василий иванович в партия он к котовский помочь что говорить на партсобрание ну как там ты вопрос быть задавать ты выучить устать...


4. Посчитайте частотность появления каждого слова в тексте и сформируйте словарь count_words

5. Укажите частоту появления слова молодец

6. Запишите в список my_stopwords слова, которые встречаются чаще 1000 раз и укажите размер этого списка. Он пригодится чуть позже.

Внимание! В поле ответа внесите через пробел два числа, первое из которых указывает частоту появления слова молодец, второе - размер списка my_stopwords, например, 100 50

In [30]:
all_words = train_processed.explode().tolist()

# Создаем словарь с подсчетом частоты каждого слова
count_words = Counter(all_words)

# Частота появления слова "молодец"
frequency_molodets = count_words.get('молодец', 0)

# Формируем список стоп-слов, встречающихся более 1000 раз
my_stopwords = [word for word, count in count_words.items() if count > 1000]

# Размер списка стоп-слов
stopwords_size = len(my_stopwords)

# Выводим результат
print(f"Частота слова 'молодец': {frequency_molodets}")
print(f"Размер списка my_stopwords: {stopwords_size}")

Частота слова 'молодец': 195
Размер списка my_stopwords: 98


- Инициализируйте объект CountVectorizer c max_features=10000.
- Обучите векторизатор на данных после применения функции preprocess_txt.
- Преобразуйте тренировочную и тестовую выборки.
- Обучите модель LogisticRegression.
- Укажите, какая метрика accuracy получилась, округлив значение до второго знака после точки.

In [38]:
def preprocess_txt(line):
    punctuation = string.punctuation  # Пунктуационные знаки
    # Удаляем знаки препинания и приводим текст к нижнему регистру
    tokens = "".join(i.lower() for i in line.strip() if i not in punctuation).split()
    
    words_lem = []
    for w in tokens:
        if w in cache:
            # Если слово в кэше, используем его нормальную форму
            words_lem.append(cache[w])
        else:
            # Лемматизируем слово и добавляем в кэш
            temp_cach = cache[w] = morph.parse(w)[0].normal_form
            words_lem.append(temp_cach)
    
    # Возвращаем текст с лемматизированными словами
    return ' '.join(words_lem)

# Применяем функцию к 10-му объекту (индекс 9)
text = df.loc[9, 'text']  # берем текст с индексом 9
preprocessed_tokens = preprocess_txt(text)

# Разделим на признаки и целевую переменную
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['rating'], test_size=0.2, random_state=1)

# Применяем функцию preprocess_txt к тренировочной и тестовой выборкам
X_train_processed = X_train.apply(preprocess_txt)
X_test_processed = X_test.apply(preprocess_txt)

# Инициализируем CountVectorizer
vectorizer = CountVectorizer(max_features=10000)

# Обучаем векторизатор и трансформируем данные
X_train_vec = vectorizer.fit_transform(X_train_processed)
X_test_vec = vectorizer.transform(X_test_processed)

# Обучаем логистическую регрессию
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Предсказание и оценка
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.68


Стоп-слова из библиотеке nltk:
- Скачайте стоп-слова для русского языка из библиотеки nltk.
- Напишите функцию del_stopwords(line, stopwords) для удаления стоп-слов, на вход функции поступает текст и список стоп-слов, на выходе функции текст без стоп-слов.
- Удалите стоп-слова из тренировочной и тестовой выборках.
- Инициализируйте объект CountVectorizer c max_features=10000.
- Обучите векторизатор на данных после применения функции preprocess_txt и после удаления стоп-слов.
- Преобразуйте тренировочную и тестовую выборки.
- Обучите модель LogisticRegression.
- Посчитайте метрику accuracy.

Стоп-слова из задания 2:
- Используйте список стоп-слов my_stopwords из задания 2, удалите их из тренировочной и тестовой выборок.
- Инициализируйте объект CountVectorizer c max_features=10000.
- Обучите векторизатор на данных после применения функции preprocess_txt и после удаления стоп-слов.
- Преобразуйте тренировочную и тестовую выборки.
- Обучите модель LogisticRegression.
- Посчитайте метрику accuracy.
- Укажите разницу в метриках, округлив её до второго знака после точки.

In [39]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
def del_stopwords(line, stopwords_list):
    return ' '.join([word for word in line.split() if word not in stopwords_list])

In [ ]:
# Предобработка и удаление nltk-стопслов
X_train_processed_nltk = X_train.apply(preprocess_txt).apply(lambda x: del_stopwords(x, russian_stopwords))
X_test_processed_nltk = X_test.apply(preprocess_txt).apply(lambda x: del_stopwords(x, russian_stopwords))

In [42]:
vectorizer_nltk = CountVectorizer(max_features=10000)
X_train_vec_nltk = vectorizer_nltk.fit_transform(X_train_processed_nltk)
X_test_vec_nltk = vectorizer_nltk.transform(X_test_processed_nltk)

model_nltk = LogisticRegression(max_iter=1000)
model_nltk.fit(X_train_vec_nltk, y_train)
y_pred_nltk = model_nltk.predict(X_test_vec_nltk)
accuracy_nltk = accuracy_score(y_test, y_pred_nltk)
print(f"Accuracy (nltk stopwords): {accuracy_nltk:.2f}")

Accuracy (nltk stopwords): 0.68


In [43]:
my_stopwords = [word for word, freq in count_words.items() if freq > 1000]

In [44]:
X_train_processed_my = X_train.apply(preprocess_txt).apply(lambda x: del_stopwords(x, my_stopwords))
X_test_processed_my = X_test.apply(preprocess_txt).apply(lambda x: del_stopwords(x, my_stopwords))

vectorizer_my = CountVectorizer(max_features=10000)
X_train_vec_my = vectorizer_my.fit_transform(X_train_processed_my)
X_test_vec_my = vectorizer_my.transform(X_test_processed_my)

model_my = LogisticRegression(max_iter=1000)
model_my.fit(X_train_vec_my, y_train)
y_pred_my = model_my.predict(X_test_vec_my)
accuracy_my = accuracy_score(y_test, y_pred_my)
print(f"Accuracy (my stopwords): {accuracy_my:.2f}")

Accuracy (my stopwords): 0.68


In [45]:
delta = round(abs(accuracy_nltk - accuracy_my), 2)
print(f"Разница метрик: {delta}")

Разница метрик: 0.0


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...


True

Целевой признак cyberbullying_type

Создайте объект класса WordNetLemmatizer

Напишите функцию preprocess_txt для предобработки текста:

Удаление хештегов и упоминание.

Удаление пунктуационных знаков (символы возьмите из модуля string).

Лемматизируйте слова.

Удалите стоп-слова из библиотеки nltk.

Можете воспользоваться следующей заготовкой.

Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=1.

Запустите функцию preprocess_txt на тренировочной и тестовой выборках.

Посчитайте кол-во слов в каждом тексте после преобразования и удалите строки, если размер равен 0.

Инициализируйте объект TfidfVectorizer c max_features=10000.

Обучите векторизатор на данных после применения функции preprocess_txt.

Преобразуйте тренировочную и тестовую выборки.

Обучите модель LogisticRegression.

Укажите, какая метрика accuracy получилась, округлив её до второго знака после точки.

In [49]:
df = pd.read_csv("D:\\ds\\cyberbullying_tweets.csv")

In [84]:
import re
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


X = df['tweet_text']  # Тексты
y = df['cyberbullying_type']  # Целевая переменная

# Создаем объект WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Функция предобработки текста
def preprocess_txt(text):
    # Удаление хештегов и упоминаний
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'@\w+', '', text)
    
    # Удаление пунктуации
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Лемматизация
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]
    
    return ' '.join(words)

# Разбиение на обучение и тест (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Применяем предобработку
X_train_processed = X_train.apply(preprocess_txt)
X_test_processed = X_test.apply(preprocess_txt)

# Удаляем пустые тексты
train_non_empty = X_train_processed.str.len() > 0
X_train_processed = X_train_processed[train_non_empty]
y_train = y_train[train_non_empty]

test_non_empty = X_test_processed.str.len() > 0
X_test_processed = X_test_processed[test_non_empty]
y_test = y_test[test_non_empty]

# Векторизация (TF-IDF)
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train_processed)
X_test_vec = vectorizer.transform(X_test_processed)

# Обучение модели
model = LogisticRegression(max_iter=1000)  # Увеличиваем max_iter для сходимости
model.fit(X_train_vec, y_train)

# Предсказание и оценка
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81


Обучите модель Word2Vec из модуля gensim:
- Возьмите размер эмбеддинга 100.
- Минимальное количество появления слова 2.
- Окно размера 5.
- Для воспроизводимости зафиксируйте параметры seed=1, workers=1.
- Выведите эмбеддинг для слова cat и укажите нулевое значение, округлив его до второго знака после точки.

In [ ]:
from gensim.models import Word2Vec

# Разбиваем предобработанные тексты на списки слов
sentences = [text.split() for text in X_train_processed]

# Обучаем Word2Vec
model_w2v = Word2Vec(
    sentences=sentences,
    vector_size=100,       # Размер эмбеддинга
    window=5,              # Размер окна
    min_count=2,           # Минимальное количество появления слова
    seed=1,                # Для воспроизводимости
    workers=1              # Количество потоков
)

# Получаем эмбеддинг для слова "cat"
try:
    embedding_cat = model_w2v.wv["cat"]
    print(f"Нулевое значение эмбеддинга 'cat': {round(embedding_cat[0], 2)}")
except KeyError:
    print("Слово 'cat' отсутствует в словаре Word2Vec.")

Нулевое значение эмбеддинга 'cat': -0.15000000596046448


Реализуйте функцию get_mean_embedding для подсчета среднего эмбеддинга для всего предложения:
- Получите размер эмбеддинга из атрибута модели.
- Суммируйте значения эмбеддингов.
- Усредните значения эмбеддингов по кол-ву слов, которые есть в модели Word2Vec.
- Получите усредненный эмбеддинг для тренировочной и тестовой выборок.
- Обучите модель LogisticRegression.
- Укажите, какая метрика accuracy получилась, округлив значение до второго знака после точки.

In [86]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def get_mean_embedding(sent, model):
    vector_size = model.vector_size  # получение размера эмбеддинга из модели
    model_res = np.zeros(vector_size)  # заготовка для усредненного эмбеддинга
    ctr = 0  # счетчик слов
    for word in sent.split():  # разбиваем предложение на слова
        if word in model.wv:  # проверяем, есть ли слово в модели
            model_res += model.wv[word]  # добавление эмбеддинга
            ctr += 1
    if ctr > 0:
        model_res /= ctr  # усреднение эмбеддинга по словам
    return model_res

# Получаем усредненные эмбеддинги для тренировочной и тестовой выборок
X_train_w2v = np.array([get_mean_embedding(text, model_w2v) for text in X_train_processed])
X_test_w2v = np.array([get_mean_embedding(text, model_w2v) for text in X_test_processed])

# Удаляем строки, где не нашлось ни одного слова из Word2Vec
train_non_empty = np.array([len(x) > 0 for x in X_train_w2v])
X_train_w2v = X_train_w2v[train_non_empty]
y_train_w2v = y_train[train_non_empty]

test_non_empty = np.array([len(x) > 0 for x in X_test_w2v])
X_test_w2v = X_test_w2v[test_non_empty]
y_test_w2v = y_test[test_non_empty]

# Обучаем LogisticRegression на эмбеддингах Word2Vec
lr_model = LogisticRegression(max_iter=1000, random_state=1)
lr_model.fit(X_train_w2v, y_train_w2v)

# Предсказание и оценка качества
y_pred_w2v = lr_model.predict(X_test_w2v)
accuracy_w2v = accuracy_score(y_test_w2v, y_pred_w2v)

print(f"Accuracy (Word2Vec + LogisticRegression): {accuracy_w2v:.2f}")

Accuracy (Word2Vec + LogisticRegression): 0.73


Реализуйте функцию tokenize для токенизации текстов с входными параметрами:
- sents - предложения.
- seq_len - длина текста.
- Пройдитесь в цикле по текстам.
- Пройдитесь в цикле по словам.
- Если слово есть в словаре, то получите его индекс и добавьте в список.
- Если у текста длина меньше seq_len, нужно ее заполнить токеном PAD.
- Запустите функцию tokenize на предобработанных (после функции preprocess_txt) тренировочной и тестовой выборках с максимальной длиной последовательности 50
- Укажите через пробел индексы последних трех слов в нулевом тренировочном тексте. Например, 20 4000 500.

In [87]:
# Создаем словарь модели с токеном PAD
vocab = {k: v+1 for k, v in model_w2v.wv.key_to_index.items()}
vocab['PAD'] = 0

def tokenize(sents, seq_len):
    # Перевод текстов в численный вид
    text_int = []
    for text in sents:  # проход по текстам
        r = []
        for word in text.split():  # проход по словам в тексте
            if word in vocab:  # если слово есть в словаре
                r.append(vocab[word])  # добавляем индекс слова
        text_int.append(r)

    # Добавление паддинга
    features = np.zeros((len(text_int), seq_len), dtype=int)
    for i, text in enumerate(text_int):
        if len(text) < seq_len:  # если размер текста меньше seq_len
            # Добавляем PAD токены в начало
            new = [vocab['PAD']] * (seq_len - len(text)) + text
        else:
            # Урезаем текст до seq_len
            new = text[:seq_len]
        features[i, :] = np.array(new)

    return features

# Применяем токенизацию с seq_len=50
X_train_tokenized = tokenize(X_train_processed, seq_len=50)
X_test_tokenized = tokenize(X_test_processed, seq_len=50)

# Получаем индексы последних трех слов в нулевом тексте
last_three_indices = X_train_tokenized[0][-3:]

print("Индексы последних трех слов:", ' '.join(map(str, last_three_indices)))

Индексы последних трех слов: 21 9037 811


Сформируйте матрицу эмбеддингов, где каждая строка будет из себя представлять эмбеддинг. Размер матрицы должен быть (16682, 100).

Преобразуйте строковое обозначение классов в порядковые числа через LabelEncoder

Создайте TensorDataset из массива индексов слов для выборки тренировочной и тестовой.

Создайте DataLoader для обучения и теста. Используйте перемешивание и batch_size=128.

Сделайте одну итерацию в цикле по DataLoader для обучения и укажите два последних индекса слов для нулевого объекта. Для воспроизводимости используйте torch.manual_seed(1)
Внимание! В поле ответа через пробел внесите два последних индекса слов для нулевого объекта, например, 2000 100

In [88]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# Формируем матрицу эмбеддингов (16682, 100)
embedding_matrix = np.zeros((len(vocab), 100))  # +1 для PAD токена
for word, idx in vocab.items():
    if word != 'PAD' and word in model_w2v.wv:
        embedding_matrix[idx] = model_w2v.wv[word]

# Преобразуем строковые классы в числовые
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Создаем TensorDataset
train_dataset = TensorDataset(torch.LongTensor(X_train_tokenized), 
                             torch.LongTensor(y_train_encoded))
test_dataset = TensorDataset(torch.LongTensor(X_test_tokenized), 
                            torch.LongTensor(y_test_encoded))

# Создаем DataLoader с перемешиванием
torch.manual_seed(1)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Одна итерация по DataLoader
for batch in train_loader:
    inputs, labels = batch
    # Получаем два последних индекса для первого объекта в батче
    last_two_indices = inputs[0][-2:].tolist()
    print(' '.join(map(str, last_two_indices)))
    break

2698 615


Создайте класс нейронной сети Classifier, который наследуется от nn.Module на классификацию на 6 классов:
- со слоем Embedding размерностью 100
- с фиксированной случайностью torch.manual_seed(1)
- с усреднением эмбеддингов слов в предложении

Укажите, какое предсказание получается у необученной нейросети на нулевом тренировочном объекте на нулевой класс.

In [89]:
import torch
from torch import nn

class Classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_out):
        super().__init__()
        torch.manual_seed(1)  # Фиксация случайности
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, n_out)
    
    def forward(self, x):
        # x shape: [batch_size, seq_len]
        embedded_words = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        embedded_sent = torch.mean(embedded_words, dim=1)  # Усреднение по словам [batch_size, embedding_dim]
        out = self.fc(embedded_sent)  # [batch_size, n_out]
        return out

# Параметры модели
vocab_size = len(vocab)  # Размер словаря (включая PAD)
embedding_dim = 100      # Размерность эмбеддингов
n_out = 6                # Количество классов

# Создаем модель
model = Classifier(vocab_size, embedding_dim, n_out)

# Получаем предсказание для нулевого объекта
with torch.no_grad():
    sample_input = torch.LongTensor(X_train_tokenized[:1])  # Берем первый объект
    prediction = model(sample_input)
    class_0_prob = torch.softmax(prediction, dim=1)[0, 0].item()

print(f"Вероятность нулевого класса: {class_0_prob:.4f}")

Вероятность нулевого класса: 0.1412


В этом задании будете обучать только один линейный слой на классификацию текстов по эмбеддингам из модели Word2Vec. Для этого замените веса в слое Embedding на веса из матрицы embedding_matrix. Укажите для этого слоя requires_grad = False

Запустите обучение:

Оптимизатор Adam со скоростью обучения 0.001.

Количество эпох 10.

Укажите, какая метрика accuracy получается на последней эпохе на тестовой выборке, округлив значение до второго знака после точки.

In [92]:
from tqdm import tqdm
from torch import nn, optim

def train_model(model, opt, train_loader, test_loader, n_epoch=100):
    loss_fn = nn.CrossEntropyLoss()
    acc_train_list, acc_test_list = [], []

    for epoch_num in range(n_epoch):
        train_loss, test_loss = 0.0, 0.0
        correct_train, correct_test = 0, 0
        total_train, total_test = 0, 0

        for data, labels in tqdm(train_loader):
            opt.zero_grad()  # обнуление градиентов
            y_predicted = model(data)  # получение предсказаний
            loss = loss_fn(y_predicted, labels)  # подсчет функции потерь

            loss.backward()  # обратный проход
            opt.step()  # шаг оптимизации

            # фиксация функции потерь и accuracy на обучении
            train_loss += loss.item()  # суммируем ошибку
            total_train += len(labels)  # суммируем кол-во объектов
            _, predicted = torch.max(y_predicted, 1)  # получаем предсказанные классы
            correct_train += (predicted == labels).sum().item()  # суммируем кол-во правильных классификаций

        # вычисляем значение функции потерь и метрик на обучении на всем train_loader
        train_acc = correct_train / total_train
        train_loss = train_loss / total_train
        acc_train_list.append(train_acc)


        for data, labels in test_loader:
            outputs = model(data)
            loss = loss_fn(outputs, labels)

            # фиксация функции потерь и accuracy на тесте
            test_loss += loss.item()  # суммируем ошибку
            total_test += len(labels)  # суммируем кол-во объектов
            _, predicted = torch.max(outputs.data, 1)  # получаем предсказанные классы
            correct_test += (predicted == labels).sum().item()  # суммируем кол-во правильных классификаций

        # вычисляем значение функции потерь и метрик на тесте на всем test_loader
        test_acc = correct_test / total_test
        test_loss = test_loss / total_test

        acc_test_list.append(test_acc)


        # печатаем результаты эпохи
        print(f"Epoch {epoch_num+1}/{n_epoch} " \
              f"Train Loss: {train_loss:.3f} - Train Acc: {train_acc:.3f} " \
              f"Test Loss: {test_loss:.3f} - Test Acc: {test_acc:.3f}")

    return model, acc_train_list, acc_test_list 

class Classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_out, embedding_matrix):
        super().__init__()
        torch.manual_seed(1)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Загружаем предобученные веса и замораживаем их
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.fc = nn.Linear(embedding_dim, n_out)
    
    def forward(self, x):
        embedded_words = self.embedding(x)
        embedded_sent = torch.mean(embedded_words, dim=1)
        return self.fc(embedded_sent)

# 2. Инициализация модели
vocab_size = len(vocab)
embedding_dim = 100
n_out = 6
model = Classifier(vocab_size, embedding_dim, n_out, embedding_matrix)

# 3. Обучение модели
optimizer = optim.Adam(model.parameters(), lr=0.001)
model, train_acc, test_acc = train_model(model, optimizer, train_loader, test_loader, n_epoch=10)

# 4. Получение accuracy на последней эпохе
final_test_accuracy = test_acc[-1]
print(f"\nFinal Test Accuracy: {final_test_accuracy:.2f}")

100%|██████████| 298/298 [00:00<00:00, 396.10it/s]


Epoch 1/10 Train Loss: 0.011 - Train Acc: 0.593 Test Loss: 0.010 - Test Acc: 0.651


100%|██████████| 298/298 [00:00<00:00, 385.01it/s]


Epoch 2/10 Train Loss: 0.009 - Train Acc: 0.674 Test Loss: 0.008 - Test Acc: 0.691


100%|██████████| 298/298 [00:00<00:00, 382.34it/s]


...

Epoch 8/10 Train Loss: 0.005 - Train Acc: 0.739 Test Loss: 0.005 - Test Acc: 0.738


100%|██████████| 298/298 [00:00<00:00, 384.05it/s]


Epoch 9/10 Train Loss: 0.005 - Train Acc: 0.739 Test Loss: 0.005 - Test Acc: 0.740


100%|██████████| 298/298 [00:00<00:00, 344.78it/s]


Epoch 10/10 Train Loss: 0.005 - Train Acc: 0.742 Test Loss: 0.005 - Test Acc: 0.741

Final Test Accuracy: 0.74


Создайте заново нейросеть Classifier.

Замените веса в слое Embedding на веса из матрицы embedding_matrix.

Оставьте значение для этого слоя requires_grad = True.

Запустите обучение:

Оптимизатор Adam со скоростью обучения 0.001.

Количество эпох 10.

Укажите, какая метрика accuracy получается на последней эпохе на тестовой выборке, округлив значение до второго знака после точки.

In [93]:
class Classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_out, embedding_matrix):
        super().__init__()
        torch.manual_seed(1)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Загружаем предобученные веса, но разрешаем их обновление
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = True  # Разрешаем обучение эмбеддингов
        self.fc = nn.Linear(embedding_dim, n_out)
    
    def forward(self, x):
        embedded_words = self.embedding(x)
        embedded_sent = torch.mean(embedded_words, dim=1)
        return self.fc(embedded_sent)

# 2. Инициализация модели
vocab_size = len(vocab)
embedding_dim = 100
n_out = 6
model = Classifier(vocab_size, embedding_dim, n_out, embedding_matrix)

# 3. Обучение модели
optimizer = optim.Adam(model.parameters(), lr=0.001)
model, train_acc, test_acc = train_model(model, optimizer, train_loader, test_loader, n_epoch=10)

# 4. Получение accuracy на последней эпохе
final_test_accuracy = test_acc[-1]
print(f"\nFinal Test Accuracy: {final_test_accuracy:.2f}")

100%|██████████| 298/298 [00:03<00:00, 92.67it/s] 


Epoch 1/10 Train Loss: 0.010 - Train Acc: 0.645 Test Loss: 0.008 - Test Acc: 0.698


100%|██████████| 298/298 [00:02<00:00, 100.85it/s]


Epoch 2/10 Train Loss: 0.006 - Train Acc: 0.733 Test Loss: 0.006 - Test Acc: 0.744


100%|██████████| 298/298 [00:03<00:00, 96.11it/s] 


...

Epoch 8/10 Train Loss: 0.003 - Train Acc: 0.856 Test Loss: 0.004 - Test Acc: 0.812


100%|██████████| 298/298 [00:02<00:00, 100.22it/s]


Epoch 9/10 Train Loss: 0.003 - Train Acc: 0.865 Test Loss: 0.003 - Test Acc: 0.819


100%|██████████| 298/298 [00:03<00:00, 98.40it/s] 


Epoch 10/10 Train Loss: 0.003 - Train Acc: 0.872 Test Loss: 0.003 - Test Acc: 0.820

Final Test Accuracy: 0.82
